In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.13.1+cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 19.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import sys
# sys.path.append('/content/drive/My Drive/PyGHetero') # 此为你的自定义模块的存储路径
sys.path.append('/content/drive/MyDrive/PyGHetero/examples/pyghetero')
sys.path.append('/content/drive/MyDrive/PyGHetero/examples')

In [4]:
# ## test 路径
# dataname_ = 'cora'
# mode_ = 'sparse'
# from pyghetero.data.load_datasets import load_dataset, load_dataset_fixed_split
# data = load_dataset_fixed_split(dataname_, mode_) 

In [5]:
!pip install wandb -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.3 MB/s eta 0:00:00


In [6]:
import wandb
wandb.finish()
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
dataname_ = 'citeseer'
mode_ = 'sparse'
EPOCH = 1000

# FAGCN

## wandb setting: search space

config

In [8]:
import pprint
sweep_config = {
    'method': 'grid' # grid search ************************** 太耗资源了
    }

metric = {
    'name': 'mean_test_acc',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

# lr = [0.01, 0.005]
# weight_decay = [5e-4, 5e-3, 1e-3]

# hidden = [16, 32, 64]
# num_layers = [2, 4, 6, 8]
# eps = [0.3, 0.4, 0.5, 0.6]
# dropout = [0.4, 0.5, 0.6]

# mode = ['dense', 'sparse']
# dataset = ['cora', 'citeseer', 'pubmed', 'chamelon', 'squirrel', 'texas', 'wisconsin', 'cornell', 'crocodile']


parameters_dict = {
    'lr': {
        # 'values': [0.01, 0.005]
        'values': [0.01]
        },
    'weight_decay': {
        # 'values': [5e-4, 5e-3, 1e-3]
        'values': [1e-3]
        },
    'hidden': {
        'values': [16, 32]
        },
    'num_layers': {
          'values': [2, 4]
        }, 
    'eps': {
          'values': [0.2, 0.3, 0.4, 0.5]
        },       
    'dropout': {
          'values': [0.5, 0.6]
        },
    }

sweep_config['parameters'] = parameters_dict


pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'dropout': {'values': [0.5, 0.6]},
                'eps': {'values': [0.2, 0.3, 0.4, 0.5]},
                'hidden': {'values': [16, 32]},
                'lr': {'values': [0.01]},
                'num_layers': {'values': [2, 4]},
                'weight_decay': {'values': [0.001]}}}


training

In [9]:
import argparse
import random
import numpy as np
import torch.nn.functional as F
from pyghetero.models.gat import GAT
from pyghetero.models.fagcn import FAGCN
from pyghetero.utils.data_utils import *
from pyghetero.data.load_datasets import load_dataset, load_dataset_fixed_split


def train_epoch(model, optimizer, data, train_mask):
    model.train()
    optimizer.zero_grad()
    z = model(data.x, data.edge_index)
    z = F.log_softmax(z, dim=1)
    loss = F.nll_loss(z[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    return loss.item()


@torch.no_grad()
def test(model, data, train_mask, val_mask, test_mask):
    model.eval()
    z = model(data.x, data.edge_index)
    z = F.log_softmax(z, dim=1)
    train_pred = z[train_mask].max(1)[1]
    train_acc = train_pred.eq(data.y[train_mask]).sum().item() / train_mask.sum().item()
    val_pred = z[val_mask].max(1)[1]
    val_acc = val_pred.eq(data.y[val_mask]).sum().item() / val_mask.sum().item()
    test_pred = z[test_mask].max(1)[1]
    test_acc = test_pred.eq(data.y[test_mask]).sum().item() / test_mask.sum().item()
    return train_acc, val_acc, test_acc

def train(config=None):
    with wandb.init(config=config):
      args = wandb.config

      data = load_dataset_fixed_split(dataname_, mode_) # set dataname and mode
      device = "cuda:0" if torch.cuda.is_available() else "cpu"
      data = data.to(device)

      print('Data:', data)

      num_nodes = data.num_nodes
      num_classes = data.y.max().item() + 1
      num_features = data.x.shape[1]

      # Training
      print('Begin training:')

      best_val_acc_multi, test_acc_multi = [], []
      print('num of masks:', data.train_mask.shape[1])
      for i in range(data.train_mask.shape[1]):
          print('MASK:', i)
          train_mask = data.train_mask[:, i]
          val_mask = data.val_mask[:, i]
          test_mask = data.test_mask[:, i]

          model = FAGCN(num_features=num_features,
                      num_hidden=args.hidden,
                      num_out=num_classes,
                      num_layers=args.num_layers,
                      eps=args.eps,
                      dropout=args.dropout).to(device)


          optimizer = torch.optim.Adam(model.parameters(),
                                      lr=args.lr,
                                      weight_decay=args.weight_decay)

          best_val_acc = best_test_acc = 0

          for epoch in range(EPOCH):
              train_loss = train_epoch(model, optimizer, data, train_mask)

              train_acc, val_acc, test_acc = test(model, data, train_mask, val_mask, test_mask)
              if val_acc > best_val_acc:
                  best_val_acc = val_acc
                  best_test_acc = test_acc

              if epoch % 200 == 0:
                  log = 'Epoch:{:04d}, Train_loss:{:.4f}, Train_acc:{:.3f}, Val_acc:{:.3f}, Test_acc:{:.3f}, Best_test_acc:{:.3f}'
                  # print(
                  #     log.format(epoch, train_loss, train_acc * 100, val_acc * 100, test_acc * 100, best_test_acc * 100))
                  # wandb.log({'epoch': epoch, 'train_loss': train_loss, 'train_acc': train_acc * 100, })

          best_val_acc_multi.append(best_val_acc)
          test_acc_multi.append(best_test_acc)

      # Process results
      best_val_acc_multi.append(np.mean(best_val_acc_multi))
      test_acc_multi.append(np.mean(test_acc_multi))
      best_val_acc_multi = (np.array(best_val_acc_multi) * 100).reshape(-1, 1)
      test_acc_multi = (np.array(test_acc_multi) * 100).reshape(-1, 1)
      result = np.around(np.concatenate((best_val_acc_multi, test_acc_multi), 1), decimals=3)

      wandb.log({'mean_best_val_acc': result[-1, 0], 'mean_test_acc': result[-1, 1], 'std_test_acc': np.std(result[0:data.train_mask.shape[1], 1])})



In [10]:
sweep_id = wandb.sweep(sweep_config, project="1_pyg_gat_"+dataname_+'_'+mode_)

# wandb.agent(sweep_id, train, count=5) # train: 一个函数
wandb.agent(sweep_id, train, count=5) # train: 一个函数

Create sweep with ID: aii42q8l
Sweep URL: https://wandb.ai/yaqiwang0/1_pyg_gat_citeseer_sparse/sweeps/aii42q8l


wandb: Agent Starting Run: rtig06f2 with config:
wandb: 	dropout: 0.5
wandb: 	eps: 0.2
wandb: 	hidden: 16
wandb: 	lr: 0.01
wandb: 	num_layers: 2
wandb: 	weight_decay: 0.001
wandb: Currently logged in as: yaqiwang (yaqiwang0). Use `wandb login --relogin` to force relogin


Processing...
Done!


Data: Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327, 10], val_mask=[3327, 10], test_mask=[3327, 10])
Begin training:
num of masks: 10
MASK: 0
MASK: 1
MASK: 2
MASK: 3
MASK: 4
MASK: 5
MASK: 6
MASK: 7
MASK: 8
MASK: 9


mean_best_val_acc,▁
mean_test_acc,▁
std_test_acc,▁
mean_best_val_acc,70.843
mean_test_acc,64.641
std_test_acc,2.12851


...

Data: Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327, 10], val_mask=[3327, 10], test_mask=[3327, 10])
Begin training:
num of masks: 10
MASK: 0
MASK: 1
MASK: 2
MASK: 3
MASK: 4
MASK: 5
MASK: 6
MASK: 7
MASK: 8
MASK: 9


mean_best_val_acc,▁
mean_test_acc,▁
std_test_acc,▁
mean_best_val_acc,71.566
mean_test_acc,65.369
std_test_acc,1.42949


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: x9fsi79f with config:
wandb: 	dropout: 0.5
wandb: 	eps: 0.3
wandb: 	hidden: 16
wandb: 	lr: 0.01
wandb: 	num_layers: 2
wandb: 	weight_decay: 0.001


Data: Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327, 10], val_mask=[3327, 10], test_mask=[3327, 10])
Begin training:
num of masks: 10
MASK: 0
MASK: 1
MASK: 2
MASK: 3
MASK: 4
MASK: 5
MASK: 6
MASK: 7
MASK: 8
MASK: 9


mean_best_val_acc,▁
mean_test_acc,▁
std_test_acc,▁
mean_best_val_acc,70.482
mean_test_acc,64.03
std_test_acc,2.15725


In [11]:
wandb.finish()

## self-doing

对每一个不同的数据集、mode，创建不同的工程，并且用wandb

In [12]:
import pprint
sweep_config = {
    'method': 'grid' # grid search ************************** 太耗资源了
    }

metric = {
    'name': 'mean_test_acc',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

# lr = [0.01, 0.005]
# weight_decay = [5e-4, 5e-3, 1e-3]

# hidden = [16, 32, 64]
# num_layers = [2, 4, 6, 8]
# eps = [0.3, 0.4, 0.5, 0.6]
# dropout = [0.4, 0.5, 0.6]

# mode = ['dense', 'sparse']
# dataset = ['cora', 'citeseer', 'pubmed', 'chamelon', 'squirrel', 'texas', 'wisconsin', 'cornell', 'crocodile']


parameters_dict = {
    'lr': {
        # 'values': [0.01, 0.005]
        'values': [0.01]
        },
    'weight_decay': {
        # 'values': [5e-4, 5e-3, 1e-3]
        'values': [1e-3]
        },
    'hidden': {
        'values': [16, 32]
        },
    'num_layers': {
          'values': [2, 4]
        }, 
    'eps': {
          'values': [0.2, 0.3, 0.4, 0.5]
        },       
    'dropout': {
          'values': [0.5, 0.6]
        },
    }

sweep_config['parameters'] = parameters_dict


pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'mean_test_acc'},
 'parameters': {'dropout': {'values': [0.5, 0.6]},
                'eps': {'values': [0.2, 0.3, 0.4, 0.5]},
                'hidden': {'values': [16, 32]},
                'lr': {'values': [0.01]},
                'num_layers': {'values': [2, 4]},
                'weight_decay': {'values': [0.001]}}}


In [13]:
import argparse
import random
import numpy as np
import torch.nn.functional as F
from pyghetero.models.gat import GAT
from pyghetero.models.fagcn import FAGCN
from pyghetero.utils.data_utils import *
from pyghetero.data.load_datasets import load_dataset, load_dataset_fixed_split


def train_epoch(model, optimizer, data, train_mask):
    model.train()
    optimizer.zero_grad()
    z = model(data.x, data.edge_index)
    z = F.log_softmax(z, dim=1)
    loss = F.nll_loss(z[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    return loss.item()


@torch.no_grad()
def test(model, data, train_mask, val_mask, test_mask):
    model.eval()
    z = model(data.x, data.edge_index)
    z = F.log_softmax(z, dim=1)
    train_pred = z[train_mask].max(1)[1]
    train_acc = train_pred.eq(data.y[train_mask]).sum().item() / train_mask.sum().item()
    val_pred = z[val_mask].max(1)[1]
    val_acc = val_pred.eq(data.y[val_mask]).sum().item() / val_mask.sum().item()
    test_pred = z[test_mask].max(1)[1]
    test_acc = test_pred.eq(data.y[test_mask]).sum().item() / test_mask.sum().item()
    return train_acc, val_acc, test_acc

In [ ]:
from IPython.display import clear_output as clear

EPOCH = 1000
for mode_ in ['sparse','dense']:
  for dataname_ in ['pubmed','texas', 'wisconsin', 'cornell','squirrel', 'chameleon','crocodile']:
    
    print(mode_)
    print(dataname_)

    ## for loop 中创建函数
    def train_in_loop(config=None):
        with wandb.init(config=config):
          args = wandb.config

          data = load_dataset_fixed_split(dataname_, mode_) # set dataname and mode
          device = "cuda:0" if torch.cuda.is_available() else "cpu"
          data = data.to(device)

          print('Data:', data)

          num_nodes = data.num_nodes
          num_classes = data.y.max().item() + 1
          num_features = data.x.shape[1]

          # Training
          print('Begin training:')

          best_val_acc_multi, test_acc_multi = [], []
          print('num of masks:', data.train_mask.shape[1])
          for i in range(data.train_mask.shape[1]):
              print('MASK:', i)
              train_mask = data.train_mask[:, i]
              val_mask = data.val_mask[:, i]
              test_mask = data.test_mask[:, i]

              model = FAGCN(num_features=num_features,
                          num_hidden=args.hidden,
                          num_out=num_classes,
                          num_layers=args.num_layers,
                          eps=args.eps,
                          dropout=args.dropout).to(device)


              optimizer = torch.optim.Adam(model.parameters(),
                                          lr=args.lr,
                                          weight_decay=args.weight_decay)

              best_val_acc = best_test_acc = 0

              for epoch in range(EPOCH):
                  train_loss = train_epoch(model, optimizer, data, train_mask)

                  train_acc, val_acc, test_acc = test(model, data, train_mask, val_mask, test_mask)
                  if val_acc > best_val_acc:
                      best_val_acc = val_acc
                      best_test_acc = test_acc

                  if epoch % 200 == 0:
                      log = 'Epoch:{:04d}, Train_loss:{:.4f}, Train_acc:{:.3f}, Val_acc:{:.3f}, Test_acc:{:.3f}, Best_test_acc:{:.3f}'
                      # print(
                      #     log.format(epoch, train_loss, train_acc * 100, val_acc * 100, test_acc * 100, best_test_acc * 100))
                      # wandb.log({'epoch': epoch, 'train_loss': train_loss, 'train_acc': train_acc * 100, })

              best_val_acc_multi.append(best_val_acc)
              test_acc_multi.append(best_test_acc)

          # Process results
          best_val_acc_multi.append(np.mean(best_val_acc_multi))
          test_acc_multi.append(np.mean(test_acc_multi))
          best_val_acc_multi = (np.array(best_val_acc_multi) * 100).reshape(-1, 1)
          test_acc_multi = (np.array(test_acc_multi) * 100).reshape(-1, 1)
          result = np.around(np.concatenate((best_val_acc_multi, test_acc_multi), 1), decimals=3)

          wandb.log({'mean_best_val_acc': result[-1, 0], 'mean_test_acc': result[-1, 1], 'std_test_acc': np.std(result[0:data.train_mask.shape[1], 1])})

    ######################################################################################
    sweep_id = wandb.sweep(sweep_config, project="1_pyg_fagcn_"+dataname_+'_'+mode_)
    wandb.agent(sweep_id, train_in_loop, count=20)
    wandb.finish()
    clear()

sparse
crocodile
Create sweep with ID: 9nhoyd22
Sweep URL: https://wandb.ai/yaqiwang0/1_pyg_fagcn_crocodile_sparse/sweeps/9nhoyd22


wandb: Agent Starting Run: 211mqkfc with config:
wandb: 	dropout: 0.5
wandb: 	eps: 0.2
wandb: 	hidden: 16
wandb: 	lr: 0.01
wandb: 	num_layers: 2
wandb: 	weight_decay: 0.001


Processing...
Done!


Data: Data(x=[11631, 128], edge_index=[2, 360040], y=[11631], train_mask=[11631, 10], val_mask=[11631, 10], test_mask=[11631, 10])
Begin training:
num of masks: 10
MASK: 0
MASK: 1
MASK: 2
MASK: 3
MASK: 4
MASK: 5
MASK: 6
MASK: 7
MASK: 8
MASK: 9


mean_best_val_acc,▁
mean_test_acc,▁
std_test_acc,▁
mean_best_val_acc,63.034
mean_test_acc,61.69
std_test_acc,0.78908


...

wandb: Agent Starting Run: w25q6wn4 with config:
wandb: 	dropout: 0.5
wandb: 	eps: 0.4
wandb: 	hidden: 16
wandb: 	lr: 0.01
wandb: 	num_layers: 4
wandb: 	weight_decay: 0.001


Data: Data(x=[11631, 128], edge_index=[2, 360040], y=[11631], train_mask=[11631, 10], val_mask=[11631, 10], test_mask=[11631, 10])
Begin training:
num of masks: 10
MASK: 0
MASK: 1
MASK: 2
MASK: 3
MASK: 4
MASK: 5
MASK: 6
MASK: 7
MASK: 8
MASK: 9


mean_best_val_acc,▁
mean_test_acc,▁
std_test_acc,▁
mean_best_val_acc,62.414
mean_test_acc,60.947
std_test_acc,0.73656


wandb: Agent Starting Run: tuulreqn with config:
wandb: 	dropout: 0.5
wandb: 	eps: 0.4
wandb: 	hidden: 32
wandb: 	lr: 0.01
wandb: 	num_layers: 2
wandb: 	weight_decay: 0.001


Data: Data(x=[11631, 128], edge_index=[2, 360040], y=[11631], train_mask=[11631, 10], val_mask=[11631, 10], test_mask=[11631, 10])
Begin training:
num of masks: 10
MASK: 0
MASK: 1
MASK: 2
